In [6]:
from app import *
from openai_tools import *
from pachong2 import *
import pyperclip as clp
from db import SqliteDb

window_width = 2400
top_banner_height = 62
bottom_window_height = 1491
hao_height = 144
hao_width = 416
hao_sub_width = 704 # 号的文章栏的宽度 貌似没啥用
article_2rd_height = 196 # 除头条外文章的高度
v_first_center = top_banner_height + hao_height / 2
h_first_center = 66

def find_my1():
    my1v = img_loc("images_mac/my1_mac.png")
    return my1v

def find_my2():
    my2v=img_loc("images2/my2_2.png")
    return my2v

def img_loc(img):
    my=CardLocation(config=CardConfig(template_file=img, bias_type=BiasType.NoBias))
    myv=my.get_location
    return myv

def moveby(mv, x, y):
    time.sleep(0.8)
    pag.moveTo(x=mv.x+x,y=mv.y+y)
    time.sleep(0.8)
    
def locate_first():
    my1v = find_my1()
    moveby(my1v, h_first_center, v_first_center)
    pag.scroll(10000000)
    time.sleep(0.3)
    pag.click()
    
def open_curr_article_to_browser():
    my1v=find_my1()
    moveby(my1v, window_width-56, 62+47)
    pag.click()
    moveby(my1v, window_width-86, 62+47+200)
    pag.click()

def copy_browser_url():
    time.sleep(1.2)
    pag.hotkey('ctrl', 'l') 
    pag.hotkey('ctrl', 'c') 
    url = clp.paste()
    time.sleep(0.3)
    pag.hotkey('ctrl', 'w') 
    return url

def fetch_url_ctt(url):
    print(url)
    if db.check_url(url):
        hao, title, desc, abst, ctt = db.read('article', columns=' hao, title, desc, abst, ctt ', where=f"url='{url}'")[0]
    else:
        hao, title, desc, ctt = get_wechat_artile_content(url)
        if hao is None:
            return None, None, None, None, None
        if ctt:
            _, abst = call_with_sys(sys_prompt, ctt[:6500], 512)
            time.sleep(20)
        else:
            _, abst = '', ''
        db.create('article', {
            'title': title,
            'hao': hao,
            'url': url,
            'desc': desc,
            'abst': abst,
            'ctt': ctt
        }) 
    print("%s : %s" % (hao, title))
    return hao, title, desc, abst, ctt

def write2file(i, hao, title, url, desc, abst, ctt):
    if i == 0:
        f_meta.write("## %s \n\n" % hao)
        f_meta.flush()
    f_meta.write("### %s \n\n %s\n\n %s\n\n" % (title, abst, url))
    f_meta.flush()

    f_ctt.write("%s : %s \n %s\n%s\n\n" % (hao, title, desc, url))
    f_ctt.write("%s\n\n" % abst)
    f_ctt.write("%s\n\n\n\n\n\n" % ctt)
    f_ctt.flush()

def click_curr_hao_all_article(num):
    my1v=find_my1()
    i = -1
    while True:
        i += 1
        if i < num:
            moveby(my1v, hao_width + 30, top_banner_height + 400 + i * article_2rd_height)
            pag.click()
            time.sleep(1.3)
            open_curr_article_to_browser()
            url = copy_browser_url()
            if url not in all_ctt:
                hao, title, desc, abst, ctt = fetch_url_ctt(url)
                all_ctt[url] = title
                if hao is not None:
                    write2file(i, hao, title, url, desc, abst, ctt)
        else:
            break
        
    if hao is not None:
        f_meta.write("<hr/>\n\n\n\n\n\n")
        f_meta.flush()

In [7]:
def wechat_article_list(hao_num, art_num):
    for i in range(hao_num + 1):
        my1v=find_my1()
        if i == 0:
            locate_first()
        elif 1 <= i <= 9:
            moveby(my1v, h_first_center, v_first_center + hao_height * i)
            pag.click()
        else:
            moveby(my1v, h_first_center, v_first_center + hao_height * 9)
            pag.click()
            pag.hotkey('down')
        time.sleep(0.3)
        click_curr_hao_all_article(art_num)

if __name__ == "__main__":
    sys_prompt = "跟据用户给出的文章提取80-120字的摘要，具体时间、人物等信息不可以省略。"
    while True:
        db = SqliteDb('test.sqlite')
        db.create_table('article', 'hao TEXT, title TEXT, url TEXT, desc TEXT, abst TEXT, ctt TEXT')

        now = datetime.now().strftime('%Y%m%d_%H%M%S')
        f_meta = open("_wechat_url" + "_" + now + ".md", "w", encoding="utf-8")
        f_ctt = open("_wechat_content" + "_" + now + ".txt", "w", encoding="utf-8")

        all_ctt = {}
        wechat_article_list(200, 5)
        f_meta.close()
        f_ctt.close()
        db.close()
        
        time.sleep(10 * 60)

https://mp.weixin.qq.com/s/fUJAQFqTzH1OTeZR87AOjg
babyquant量化研究 : 血泪控诉恩将仇报白眼狼量化猎头umi
https://mp.weixin.qq.com/s/U55LqLbQLxRrgGT7incNQg
babyquant量化研究 : 第20期量化与高频研究课程（Python，6月24日更新，实盘再创新高）
https://mp.weixin.qq.com/s/tip_FGFCTZqHJHuzB5A1Bg
babyquant量化研究 : 第13期数字货币量化研究课程（Python）
https://mp.weixin.qq.com/s/Gpq9Pi0dfR8yyMn-ucxctw
https://api.openai-proxy.com/v1
雷峰网 : 抖音外卖被曝全年目标仅完成1.5%；QQ可使用微信账号登录；暴雪区别对待中国大陆玩家惹众怒｜雷峰早报
https://mp.weixin.qq.com/s/YSDTufEeFTI4T3-5V_gANA
https://api.openai-proxy.com/v1
雷峰网 : 马斯克约架扎克伯格线下“铁笼大战”；谷歌新广告嘲笑iPhone过时；AI新图灵测试：将10万美元变成100万美元丨雷峰早报
https://mp.weixin.qq.com/s/xq700NcVktz_ER28_iwXHg
https://api.openai-proxy.com/v1
NLP日志录 : LangChain Cheatsheet：LangChain参考代码库
https://mp.weixin.qq.com/s/au5210oD6K0B6EUpfTAv5Q
https://api.openai-proxy.com/v1
NLP日志录 : 使用编码工具
https://mp.weixin.qq.com/s/EscXWBLM09bgfgfUT66C9Q
https://api.openai-proxy.com/v1


APIError: HTTP code 504 from API (<html>
<head><title>504 Gateway Time-out</title></head>
<body>
<center><h1>504 Gateway Time-out</h1></center>
<hr><center>openresty</center>
</body>
</html>
)

In [ ]:
# def click_curr_hao_all_article(num):
#     my1v=find_my1()
#     i = 0

#     moveby(my1v, 290, 220 + i * 100)
#     pag.click()
#     time.sleep(1.3)
#     open_curr_article_to_browser()
#     url = copy_browser_url()
#     if url in all_ctt:
#         return 

#     hao, title, desc, ctt = get_wechat_artile_content(url)
#     if hao is not None:
#         f_meta.write("## %s \n\n" % hao)
#         f_meta.flush()

#     while True:
#         if i >= num:
#             break
#         moveby(my1v, 290, 220 + i * 120)
#         pag.click()
#         time.sleep(1.3)
#         open_curr_article_to_browser()
#         url = copy_browser_url()
#         if url in all_ctt:
#             i += 1
#             continue

#         all_ctt[url] = title

#         if db.check_url(url):
#             i += 1
#             hao, title, desc, abst, ctt = db.read('article', columns=' hao, title, desc, abst, ctt ', where=f"url='{url}'")[0]
#         else:
#             hao, title, desc, abst, ctt = fetch_url_ctt(url)
#             if hao is None:
#                 i += 1
#                 continue
#             db.create('article', {
#                 'title': title,
#                 'hao': hao,
#                 'url': url,
#                 'desc': desc,
#                 'abst': abst,
#                 'ctt': ctt
#             })

#         f_meta.write("### %s \n\n %s\n\n %s\n" % (title, abst, url))
#         f_meta.flush()
        
#         f_ctt.write("%s : %s \n %s\n%s\n\n" % (hao, title, desc, url))
#         f_ctt.write("%s\n\n" % abst)
#         f_ctt.write("%s\n\n\n\n\n\n" % ctt)
#         f_ctt.flush()
#         i += 1
        

#     if hao is not None:
#         f_meta.write("<hr/>\n\n\n\n\n\n")
#         f_meta.flush()

In [ ]:


# from app import *
# from openai_tools import *
# from pachong2 import *
# import pyperclip as clp

# all_ctt = {}
# sys_prompt = "跟据用户给出的文章提取80-120字的摘要，具体时间、人物、地点等信息不可以省略。"

# # h_plus_hao = -520
# # h_3point_right = 70
# # v_3point_down = 60
# # v_3point_down_down = v_3point_down + 120


    


# def copy_browser_url():
#     time.sleep(1.2)

#     pag.hotkey('ctrl', 'l') 
#     pag.hotkey('ctrl', 'c') 
#     url = clp.paste()
#     print("get url: " + url)
#     time.sleep(1.4)
#     pag.hotkey('ctrl', 'w') 
#     return url

# def click_curr_hao_all_article(num):
#     my1v=find_my1()
#     i = 0
#     curr_hao = None
#     while True:
#         if i >= num:
#             break
#         moveby(my1v, -180, 220+i*100)
#         pag.click()
#         time.sleep(1.8)
#         open_curr_article_to_browser()
#         url = copy_browser_url()
#         if url not in all_ctt:
#             hao, title, desc, abst, ctt = fetch_url_ctt(url)
#             all_ctt[url] = title
#             if curr_hao is None:
#                 curr_hao = hao
#                 f_meta.write("## %s \n\n" % hao)
#             else:
#                 f_meta.write("### %s \n\n %s\n\n %s\n<hr/>\n\n" % (title, abst, url))
#             f_meta.flush()
            
#             f_ctt.write("%s : %s : %s: %s\n\n" % (hao, title, desc, url))
#             f_ctt.write("%s\n\n" % abst)
#             f_ctt.write("%s\n\n\n\n\n\n" % ctt)
#             f_ctt.flush()
#             time.sleep(15)
#         else:
#             i += 1
#             continue
#     if curr_hao is not None:
#         f_meta.write("\n\n\n\n\n")
#         f_meta.flush()

# def fetch_url_ctt(url):
#     hao, title, desc, ctt = get_wechat_artile_content(url)
#     _, abst = call_with_sys(sys_prompt, ctt[:6500], 512)
#     print("%s : %s : %s" % (hao, title, url))
#     return hao, title, desc, abst, ctt   


# def gooo():
#     pass